In [2]:
import pandas as pd
import os


In [66]:
basePath = '../data/fault prediction'

res = dict(A = dict(), B = dict(), C = dict())

for root, dirs, files in os.walk(basePath):
    for file in files:
        AB = 'A' if '\\A' in root else 'B' if '\\B' in root else 'C' if '\\C' in root else None
        fileName = file.split('.')[0]
        if file.endswith('.csv'):
            fp = os.path.join(root, file)
        else:
            continue
        if file in ['event_info.csv','feature_description.csv']:
            df = pd.read_csv(fp,sep = ';')
            df = df.replace('�','°')
            res[AB][fileName] = df
        else:
            with open(fp, 'r') as f:
                header = f.readline()
            res[AB][fileName] = set(header.split(';'))


In [67]:
root

'../data/fault prediction\\C\\datasets'

In [68]:
summary = pd.DataFrame()
for windFarmID, _d in res.items():
    for filename, items in _d.items():
        if not filename.isdigit():
            continue
        summary = pd.concat([summary, pd.DataFrame({'windFarmID': windFarmID, 'filename': filename, 'items': [items]})])

summary = summary.explode('items')


In [69]:
summary.groupby(['windFarmID', 'filename']).agg(items = ('items',lambda s: s.values), nitems = ('items','nunique')) 

items  nitems
windFarmID filename                                                           
A          0         [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           10        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           13        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           14        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           17        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           22        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           24        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           25        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           26        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           3         [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           38        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           40        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           42        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           45        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           51        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           68        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           69        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           71        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           72        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           73        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           84        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
           92        [sensor_18_min, sensor_21_avg, sensor_12_avg, ...      86
B          19        [sensor_34_max, reactive_power_11_min, sensor_...     257
           2         [sensor_34_max, reactive_power_11_min, sensor_...     257
           21        [sensor_34_max, reactive_power_11_min, sensor_...     257
           23        [sensor_34_max, reactive_power_11_min, sensor_...     257
           27        [sensor_34_max, reactive_power_11_min, sensor_...     257
           34        [sensor_34_max, reactive_power_11_min, sensor_...     257
           52        [sensor_34_max, reactive_power_11_min, sensor_...     257
           53        [sensor_34_max, reactive_power_11_min, sensor_...     257
           7         [sensor_34_max, reactive_power_11_min, sensor_...     257
           74        [sensor_34_max, reactive_power_11_min, sensor_...     257
           77        [sensor_34_max, reactive_power_11_min, sensor_...     257
           82        [sensor_34_max, reactive_power_11_min, sensor_...     257
           83        [sensor_34_max, reactive_power_11_min, sensor_...     257
           86        [sensor_34_max, reactive_power_11_min, sensor_...     257
           87        [sensor_34_max, reactive_power_11_min, sensor_...     257
C          1         [sensor_187_max, sensor_136_max, sensor_21_avg...     957
           11        [sensor_187_max, sensor_136_max, sensor_21_avg...     957
           12        [sensor_187_max, sensor_136_max, sensor_21_avg...     957
           15        [sensor_187_max, sensor_136_max, sensor_21_avg...     957
           16        [sensor_187_max, sensor_136_max, sensor_21_avg...     957
           18        [sensor_187_max, sensor_136_max, sensor_21_avg...     957
           20        [sensor_187_max, sensor_136_max, sensor_21_avg...     957

In [70]:
summary.groupby(['windFarmID']).agg(items = ('items',lambda s: s.values), nitems = ('items','nunique')) 


,items,nitems
windFarmID,,
A,"[sensor_18_min, sensor_21_avg, sensor_12_avg, ...",86
B,"[sensor_34_max, reactive_power_11_min, sensor_...",257
C,"[sensor_187_max, sensor_136_max, sensor_21_avg...",957


In [98]:
# read in data from wind farm A
path = '../data/fault prediction/A/datasets'
files = os.listdir(path)
filepaths = [os.path.join(path, file) for file in files]
dfs = [pd.read_csv(fp, sep = ';') for fp in filepaths]

for _,df in enumerate(dfs):
    df['fileName'] = int (files[_].replace('.csv',''))
    

In [ ]:
pd.options.display.max_columns = None
faultData = pd.concat(dfs,ignore_index=True)


In [106]:
events = res['A']['event_info'].copy()
events['timeRange'] = events.apply(lambda row: pd.date_range(start=row['event_start'], end=row['event_end'], freq=pd.Timedelta(minutes=10)), axis=1)
events = events.explode('timeRange')


In [119]:
merged = (
    faultData.rename(columns = {'time_stamp':'timeRange','fileName':'event_id'})
    .assign(timeRange = lambda df: pd.to_datetime(df['timeRange']))
    .merge(events, on = ['timeRange','event_id'], how = 'left')
)
merged

,timeRange,asset_id,id,train_test,status_type_id,sensor_0_avg,sensor_1_avg,sensor_2_avg,wind_speed_3_avg,wind_speed_4_avg,wind_speed_3_max,wind_speed_3_min,wind_speed_3_std,sensor_5_avg,sensor_5_max,sensor_5_min,sensor_5_std,sensor_6_avg,sensor_7_avg,sensor_8_avg,sensor_9_avg,sensor_10_avg,sensor_11_avg,sensor_12_avg,sensor_13_avg,sensor_14_avg,sensor_15_avg,sensor_16_avg,sensor_17_avg,sensor_18_avg,sensor_18_max,sensor_18_min,sensor_18_std,sensor_19_avg,sensor_20_avg,sensor_21_avg,sensor_22_avg,sensor_23_avg,sensor_24_avg,sensor_25_avg,sensor_26_avg,reactive_power_27_avg,reactive_power_27_max,reactive_power_27_min,reactive_power_27_std,reactive_power_28_avg,reactive_power_28_max,reactive_power_28_min,reactive_power_28_std,power_29_avg,power_29_max,power_29_min,power_29_std,power_30_avg,power_30_max,power_30_min,power_30_std,sensor_31_avg,sensor_31_max,sensor_31_min,sensor_31_std,sensor_32_avg,sensor_33_avg,sensor_34_avg,sensor_35_avg,sensor_36_avg,sensor_37_avg,sensor_38_avg,sensor_39_avg,sensor_40_avg,sensor_41_avg,sensor_42_avg,sensor_43_avg,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,sensor_52_avg,sensor_52_max,sensor_52_min,sensor_52_std,sensor_53_avg,event_id,event_label,event_start,event_start_id,event_end,event_end_id,event_description
0,2021-08-03 06:10:00,0,0,train,0,22.0,302.9,129.4,1.7,1.7,11.7,0.5,0.7,24.0,24.0,23.9,0.0,31.0,36.0,34.0,35.0,33.0,39.0,41.0,32.0,31.0,34.0,34.0,34.0,35.3,68.3,0.0,22.1,27.0,32.0,33.0,0.9,6.9,5.9,7.9,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000049,0.012976,0.0,0.000683,-0.003756,-0.003268,-0.012732,0.001854,-0.001463,-2.2,-19.2,2.8,398.9,397.9,394.3,33.0,33.0,35.0,56.0,64.0,79.0,41.0,173.5,30.0,-1280.0,0.0,0.0,-496.0,0.0,0.0,-1280.0,-496.0,0.0,0.0,0.0,0.0,26.0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-03 06:20:00,0,1,train,0,22.0,307.1,133.6,1.7,1.7,8.3,0.4,0.7,24.0,24.1,23.9,0.0,31.0,36.0,34.0,35.0,33.0,38.0,41.0,32.0,31.0,34.0,34.0,34.0,0.0,0.0,0.0,0.0,27.0,32.0,33.0,0.9,6.9,5.8,7.9,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003756,-0.003268,-0.012927,0.001805,-0.001415,-2.2,-17.0,2.7,398.1,397.5,393.7,33.0,33.0,34.0,56.0,64.0,79.0,41.0,173.5,30.0,-1278.0,0.0,0.0,-490.0,0.0,0.0,-1278.0,-490.0,0.0,0.0,0.0,0.0,25.0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-03 06:30:00,0,2,train,0,22.0,340.6,167.1,0.9,0.9,5.9,0.4,0.4,24.0,24.0,20.7,0.1,31.0,36.0,34.0,35.0,33.0,38.0,41.0,32.0,31.0,34.0,34.0,34.0,2.8,17.6,0.0,5.8,27.0,32.0,33.0,1.0,6.9,6.3,8.8,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003951,-0.003268,-0.013366,0.001805,-0.001415,-2.2,-20.4,2.7,397.4,396.8,393.0,33.0,32.0,34.0,56.0,64.0,79.0,41.0,173.5,30.0,-1356.0,0.0,0.0,-490.0,0.0,0.0,-1356.0,-490.0,0.0,0.0,0.0,0.0,25.0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-03 06:40:00,0,3,train,0,22.0,124.4,-49.1,1.5,1.5,7.1,0.6,0.3,24.0,24.1,23.9,0.0,30.0,36.0,34.0,35.0,33.0,38.0,41.0,32.0,31.0,34.0,34.0,34.0,0.4,11.2,0.0,1.9,26.0,32.0,33.0,0.9,6.8,5.8,7.8,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003707,-0.003268,-0.012927,0.001756,-0.001463,-2.3,-20.5,2.7,400.5,400.1,396.3,32.0,32.0,34.0,55.0,64.0,79.0,41.0,173.5,29.0,-1274.0,0.0,0.0,-509.0,0.0,0.0,-1274.0,-509.0,0.0,0.0,0.0,0.0,26.0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-03 06:50:00,0,4,train,0,22.0,66.2,-107.3,1.0,1.0,2.1,0.4,0.3,24.0,24.1,23.9,0.0,30.0,36.0,34.0,35.0,33.0,38.0,41.0,31.0,30.0,34.0,34.0,34.0,0.0,0.0,0.0,0.0,26.0,32.0,33.0,0.9,6.9,5.8,7.9,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003756,-0.003268,-0.012927,0.001756,-0.001463,-2.1,-19.8,2.7,399.0,398.4,394.6,32.0,32.0,34.0,55.0,63.0,79.0,41.0,173.5,29.0,-1284.0,0.0,0.0,-499.0,0.0,0.0,-1284.0,-499.0,0.0,0.0,0.0,0.0,26.0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [118]:
faultData

,time_stamp,asset_id,id,train_test,status_type_id,sensor_0_avg,sensor_1_avg,sensor_2_avg,wind_speed_3_avg,wind_speed_4_avg,wind_speed_3_max,wind_speed_3_min,wind_speed_3_std,sensor_5_avg,sensor_5_max,sensor_5_min,sensor_5_std,sensor_6_avg,sensor_7_avg,sensor_8_avg,sensor_9_avg,sensor_10_avg,sensor_11_avg,sensor_12_avg,sensor_13_avg,sensor_14_avg,sensor_15_avg,sensor_16_avg,sensor_17_avg,sensor_18_avg,sensor_18_max,sensor_18_min,sensor_18_std,sensor_19_avg,sensor_20_avg,sensor_21_avg,sensor_22_avg,sensor_23_avg,sensor_24_avg,sensor_25_avg,sensor_26_avg,reactive_power_27_avg,reactive_power_27_max,reactive_power_27_min,reactive_power_27_std,reactive_power_28_avg,reactive_power_28_max,reactive_power_28_min,reactive_power_28_std,power_29_avg,power_29_max,power_29_min,power_29_std,power_30_avg,power_30_max,power_30_min,power_30_std,sensor_31_avg,sensor_31_max,sensor_31_min,sensor_31_std,sensor_32_avg,sensor_33_avg,sensor_34_avg,sensor_35_avg,sensor_36_avg,sensor_37_avg,sensor_38_avg,sensor_39_avg,sensor_40_avg,sensor_41_avg,sensor_42_avg,sensor_43_avg,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,sensor_52_avg,sensor_52_max,sensor_52_min,sensor_52_std,sensor_53_avg,fileName
0,2021-08-03 06:10:00,0,0,train,0,22.0,302.9,129.4,1.7,1.7,11.7,0.5,0.7,24.0,24.0,23.9,0.0,31.0,36.0,34.0,35.0,33.0,39.0,41.0,32.0,31.0,34.0,34.0,34.0,35.3,68.3,0.0,22.1,27.0,32.0,33.0,0.9,6.9,5.9,7.9,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000049,0.012976,0.0,0.000683,-0.003756,-0.003268,-0.012732,0.001854,-0.001463,-2.2,-19.2,2.8,398.9,397.9,394.3,33.0,33.0,35.0,56.0,64.0,79.0,41.0,173.5,30.0,-1280.0,0.0,0.0,-496.0,0.0,0.0,-1280.0,-496.0,0.0,0.0,0.0,0.0,26.0,0
1,2021-08-03 06:20:00,0,1,train,0,22.0,307.1,133.6,1.7,1.7,8.3,0.4,0.7,24.0,24.1,23.9,0.0,31.0,36.0,34.0,35.0,33.0,38.0,41.0,32.0,31.0,34.0,34.0,34.0,0.0,0.0,0.0,0.0,27.0,32.0,33.0,0.9,6.9,5.8,7.9,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003756,-0.003268,-0.012927,0.001805,-0.001415,-2.2,-17.0,2.7,398.1,397.5,393.7,33.0,33.0,34.0,56.0,64.0,79.0,41.0,173.5,30.0,-1278.0,0.0,0.0,-490.0,0.0,0.0,-1278.0,-490.0,0.0,0.0,0.0,0.0,25.0,0
2,2021-08-03 06:30:00,0,2,train,0,22.0,340.6,167.1,0.9,0.9,5.9,0.4,0.4,24.0,24.0,20.7,0.1,31.0,36.0,34.0,35.0,33.0,38.0,41.0,32.0,31.0,34.0,34.0,34.0,2.8,17.6,0.0,5.8,27.0,32.0,33.0,1.0,6.9,6.3,8.8,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003951,-0.003268,-0.013366,0.001805,-0.001415,-2.2,-20.4,2.7,397.4,396.8,393.0,33.0,32.0,34.0,56.0,64.0,79.0,41.0,173.5,30.0,-1356.0,0.0,0.0,-490.0,0.0,0.0,-1356.0,-490.0,0.0,0.0,0.0,0.0,25.0,0
3,2021-08-03 06:40:00,0,3,train,0,22.0,124.4,-49.1,1.5,1.5,7.1,0.6,0.3,24.0,24.1,23.9,0.0,30.0,36.0,34.0,35.0,33.0,38.0,41.0,32.0,31.0,34.0,34.0,34.0,0.4,11.2,0.0,1.9,26.0,32.0,33.0,0.9,6.8,5.8,7.8,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003707,-0.003268,-0.012927,0.001756,-0.001463,-2.3,-20.5,2.7,400.5,400.1,396.3,32.0,32.0,34.0,55.0,64.0,79.0,41.0,173.5,29.0,-1274.0,0.0,0.0,-509.0,0.0,0.0,-1274.0,-509.0,0.0,0.0,0.0,0.0,26.0,0
4,2021-08-03 06:50:00,0,4,train,0,22.0,66.2,-107.3,1.0,1.0,2.1,0.4,0.3,24.0,24.1,23.9,0.0,30.0,36.0,34.0,35.0,33.0,38.0,41.0,31.0,30.0,34.0,34.0,34.0,0.0,0.0,0.0,0.0,26.0,32.0,33.0,0.9,6.9,5.8,7.9,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.003756,-0.003268,-0.012927,0.001756,-0.001463,-2.1,-19.8,2.7,399.0,398.4,394.6,32.0,32.0,34.0,55.0,63.0,79.0,41.0,173.5,29.0,-1284.0,0.0,0.0,-499.0,0.0,0.0,-1284.0,-499.0,0.0,0.0,0.0,0.0,26.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196742,2020-04-15 09:20:00,11,54062,prediction,5,15.0,117.8,4.8,2.7,2.7,6.2,1.0,0.9,24.0,24.0,23.9,0.0,25.0,32